In [1]:
import os
import json

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [69]:
MODEL = "ft:gpt-4.1-nano-2025-04-14:personal:nova-food-classifier-try3:BzwTjxeh"

In [54]:
cgfp = pd.read_excel("~/Box/dsi-core/11th-hour/good-food-purchasing/Excel from CGFP 1/GFPP Sample Purchase Upload_6.10.25 (1).xlsx", "3. Items to Research")

/Users/jpivarski/miniforge3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [55]:
len(cgfp)

622

In [56]:
cgfp.columns

Index(['Product Identifier', 'Center Vendor ID', 'Vendor Item Number',
       'Manufacturer Code', 'Product GTIN or UPC', 'Vendor', 'Producer',
       'Processor', 'Brand Name', 'Product Type', 'Food Product Category',
       'Primary Food Product Category', 'Product Name', 'Level of Processing',
       'Whole Grain or Whole Grain Rich', 'Grain Based', 'Red Meat',
       'Healthy Beverage', 'Sodium Level', 'Energy Density',
       'Environmental Sustainability Certifications',
       'Valued Workforce Certifications', 'Animal Welfare Certifications',
       'Product Notes', 'Source Link', 'Entered By', 'Entered Date',
       'Reviewed By', 'Reviewed Date', 'Review Notes'],
      dtype='object')

In [57]:
minicgfp = cgfp[cgfp["Product GTIN or UPC"].notna()][["Product GTIN or UPC", "Vendor", "Brand Name", "Product Name", "Product Type", "Food Product Category", "Level of Processing"]]

In [58]:
minicgfp

,Product GTIN or UPC,Vendor,Brand Name,Product Name,Product Type,Food Product Category,Level of Processing
9,2.754116e+10,Ace Endico,NIAGARA,"water, ss",WATER @ SPRING 8OZ MA & NJ P* (831673),Beverages,Whole/Minimally Processed
10,2.754101e+10,Ace Endico,NIAGARA,"water, ss",WATER PURIFIED 16.9OZ PLAS P* (832129),Beverages,Whole/Minimally Processed
11,1.005260e+13,Ace Endico,PACIFIC FOODS,"plant milk, soy",MILK ULTRA SOY ORIGINAL (690776),Beverages,Ultra-Processed
13,8.425327e+09,Ace Endico,RICE DREAM,"plant milk, rice",MILK RICE ORIGINAL S* (690817),Beverages,Ultra-Processed
14,2.529300e+10,Ace Endico,SILK,"plant milk, soy, flavored",MILK SOY VERY VANILLA ASEPTIC (690799),Beverages,Ultra-Processed
...,...,...,...,...,...,...,...
518,6.014592e+11,Ace Endico,NaN,"broccoli, cut, frozen, ss",BROCCOLI FLORETS FRZ (452040),Vegetables,Whole/Minimally Processed
519,6.014592e+11,Ace Endico,NaN,pea,PEAS (453750),Vegetables,Whole/Minimally Processed
520,6.007864e+13,Ace Endico,NaN,"vegetable, blend",MIX VEGETABLE (454560),Vegetables,Whole/Minimally Processed
596,4.120682e+10,Ace Endico,SEABROOK FARMS,"corn, cut",CORN CUT DOMESTIC 20LB (453305),Vegetables,Whole/Minimally Processed


In [59]:
minicgfp["Product GTIN or UPC"].dtype

dtype('float64')

In [77]:
minicgfp["Level of Processing"].value_counts()

Level of Processing
Ultra-Processed              57
Whole/Minimally Processed    30
Culinary Ingredient          11
Moderately Processed         11
Name: count, dtype: int64

In [60]:
usda = pd.read_csv("~/Box/dsi-core/11th-hour/good-food-purchasing/branded_food.csv")

/var/folders/gd/cy7hrgys1v3ddr2jg8r4d2gr0000gn/T/ipykernel_74369/1605070333.py:1: DtypeWarning: Columns (2,3,4,6,12,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  usda = pd.read_csv("~/Box/dsi-core/11th-hour/good-food-purchasing/branded_food.csv")


In [61]:
len(usda)

1977398

In [62]:
usda.columns

Index(['fdc_id', 'brand_owner', 'brand_name', 'subbrand_name', 'gtin_upc',
       'ingredients', 'not_a_significant_source_of', 'serving_size',
       'serving_size_unit', 'household_serving_fulltext',
       'branded_food_category', 'data_source', 'package_weight',
       'modified_date', 'available_date', 'market_country',
       'discontinued_date', 'preparation_state_code', 'trade_channel',
       'short_description', 'material_code'],
      dtype='object')

In [63]:
miniusda = usda[["gtin_upc", "brand_owner", "brand_name", "branded_food_category", "ingredients"]]
miniusda["gtin_upc"] = miniusda["gtin_upc"].str.replace("-", "").str.replace(" ", "").str.replace("`", "").str.replace("X", "").str.replace(">", "").apply(lambda x: {
    "NIELSENUK0002": np.nan,
    "OldCountryStore": np.nan,
    "JARLSBERG": np.nan,
    "BOOST": np.nan,
    "HAPPYKIDORGANICS": np.nan,
}.get(x, x)).astype(np.float64)
miniusda = miniusda.query("gtin_upc.notna()")

/var/folders/gd/cy7hrgys1v3ddr2jg8r4d2gr0000gn/T/ipykernel_74369/3596894651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miniusda["gtin_upc"] = miniusda["gtin_upc"].str.replace("-", "").str.replace(" ", "").str.replace("`", "").str.replace("X", "").str.replace(">", "").apply(lambda x: {


In [64]:
miniusda

,gtin_upc,brand_owner,brand_name,branded_food_category,ingredients
32768,8.197530e+11,Mr. Tortilla,NaN,Mexican Dinner Mixes,"WATER, OAT FIBER, VITAL WHEAT GLUTEN, ORGANIC ..."
32769,8.282800e+11,Fiscalini Cheese Company LP,NaN,Cheese,"PASTEURIZED COW'S MILK, CHEESE CULTURES, ENZYM..."
32770,4.175522e+10,"Langer Juice Company, Inc.",NaN,Iced & Bottle Tea,"BREWED ORGANIC HIBISCUS TEA (FILTERED WATER, O..."
32771,7.339959e+11,"World's Fare, LLC",NaN,Frozen Fruit & Fruit Juice Concentrates,"MANGO, DARK CHOCOLATE COATING (CANE SUGAR, COC..."
32772,8.849830e+11,Clasen Quality Coatings Inc,NaN,Chocolate,"CHOCOLATE LIQUOR*, INULIN, ERYTHRITOL, COCOA B..."
...,...,...,...,...,...
1900539,9.542445e+09,Lindt & Sprungli (Schweiz) AG,LINDT,Chocolate,"DARK SWEET CHOCOLATE [SUGAR, CHOCOLATE, COCOA ..."
1900540,9.542446e+09,Lindt & Sprungli (Schweiz) AG,LINDT,Chocolate,"DARK SWEET CHOCOLATE [SUGAR, CHOCOLATE, COCOA ..."
1900541,9.800007e+09,"Ferrero U.S.A., Incorporated",TIC TAC,Chewing Gum & Mints,"SUGAR*, MALTODEXTRIN, GUM ARABIC, RICE STARCH,..."
1900542,9.800520e+09,"Ferrero U.S.A., Incorporated",KINDER,Candy,"KINDER MILK CHOCOLATE - MILK CHOCOLATE (SUGAR,..."


In [65]:
miniusda["gtin_upc"].dtype

dtype('float64')

In [66]:
merged = minicgfp.merge(miniusda, left_on="Product GTIN or UPC", right_on="gtin_upc")

In [80]:
merged["Level of Processing"] = merged["Level of Processing"].map({
    "Whole/Minimally Processed": 1,
    "Culinary Ingredient": 2,
    "Moderately Processed": 3,
    "Ultra-Processed": 4,
})

In [81]:
merged

,Product GTIN or UPC,Vendor,Brand Name,Product Name,Product Type,Food Product Category,Level of Processing,gtin_upc,brand_owner,brand_name,branded_food_category,ingredients
0,2.529300e+10,Ace Endico,SILK,"plant milk, soy, flavored",MILK SOY VERY VANILLA ASEPTIC (690799),Beverages,4,2.529300e+10,"Danone US, LLC",NaN,Plant Based Milk,"SOYMILK (FILTERED WATER, SOYBEANS), CANE SUGAR..."
1,2.370004e+10,Ace Endico,TYSON,"chicken, sausage, patty",CHIC SAUSAGE PATTY 1.43OZ S* (326324),Chicken,4,2.370004e+10,Tyson Foods Inc.,Tyson,Meat/Poultry/Other Animals Sausages Prepared/...,"Boneless chicken, contains 2% or less of the f..."
2,2.370004e+10,Ace Endico,TYSON,"chicken, drumstick, seasoned",CHIC DRUMSTICK GLAZED (328572),Chicken,4,2.370004e+10,Tyson Foods Inc.,Tyson,Meat/Poultry/Other Animals Prepared/Processed,"Chicken drumsticks, water, contains 2% or less..."
3,3.010092e+10,Ace Endico,KELLOGG'S,"cracker, graham",CRACKER GRAHAM ORIG 3PK (638893),Condiments & Snacks,4,3.010092e+10,Kellogg Company US,NaN,Biscuits/Cookies (Shelf Stable),"GRAHAM FLOUR, ENRICHED FLOUR (WHEAT FLOUR, NIA..."
4,3.010092e+10,Ace Endico,KELLOGG'S,"cracker, graham",CRACKER GRAHAM ORIG 3PK (638893),Condiments & Snacks,4,3.010092e+10,Kellogg Company US,Kellogg's,Biscuits/Cookies,"Whole wheat flour, enriched flour (wheat flour..."
5,3.010092e+10,Ace Endico,KELLOGG'S,"cracker, graham",CRACKER GRAHAM ORIG 3PK (638893),Condiments & Snacks,4,3.010092e+10,Kellogg Company US,Kellogg's,Biscuits/Cookies,"Whole wheat flour, enriched flour (wheat flour..."
6,3.010092e+10,Ace Endico,KELLOGG'S,"cracker, graham",CRACKER GRAHAM ORIG 3PK (638893),Condiments & Snacks,4,3.010092e+10,Kellogg Company US,Kellogg's,Biscuits/Cookies,"Whole wheat flour, enriched flour (wheat flour..."
7,1.410018e+10,Ace Endico,PEPPERIDGE FAR,"cracker, whole grain rich",CRACKER GOLDFISH WG CHEDDAR (638724),Condiments & Snacks,4,1.410018e+10,Pepperidge Farm Inc.,PEPPERIDGE FARM,Biscuits/Cookies,"MADE WITH SMILES AND WHOLE WHEAT FLOUR, ENRICH..."
8,1.340952e+10,Ace Endico,SWEET BABY RAY,"sauce, barbecue",SAUCE BBQ ORIGINAL (747151),Condiments & Snacks,4,1.340952e+10,Sweet Baby Rays Inc.,NaN,"Ketchup, Mustard, BBQ & Cheese Sauce","HIGH FRUCTOSE CORN SYRUP, DISTILLED VINEGAR, T..."
9,1.600015e+10,Ace Endico,GENERAL MILLS,"cereal, rice, whole grain rich, ss",CEREAL CUP CINNAMON CHEX (626054),Grain Products,4,1.600015e+10,"General Mills, Inc.",NaN,Cereal,"WHOLE GRAIN RICE, RICE, SUGAR, CANOLA OIL, SAL..."


In [82]:
model_nova = []
for ingredients in merged["ingredients"]:
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {OPENAI_API_KEY}",
        },
        json={
            "model": MODEL,
            "messages": [
                {"role": "system", "content": """
Your job is to identify a food product's NOVA classification, given its ingredient lists, as one of the four following JSON objects (with no whitespace):
* `{"nova_group":1}` for unprocessed or minimally processed foods
* `{"nova_group":2}` for processed culinary ingredients
* `{"nova_group":3}` for processed foods
* `{"nova_group":4}` for ultra-processed foods
""".strip()},
                {"role": "user", "content": ingredients},
            ],
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "nova_classification",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "nova_group": {
                                "type": "integer",
                                "enum": [1, 2, 3, 4],
                            },
                        },
                        "required": ["nova_group"],
                        "additionalProperties": False,
                    },
                },
            },
        },
    )
    model_nova.append(json.loads(response.json()["choices"][0]["message"]["content"])["nova_group"])

merged["model_nova"] = model_nova

In [83]:
merged.columns

Index(['Product GTIN or UPC', 'Vendor', 'Brand Name', 'Product Name',
       'Product Type', 'Food Product Category', 'Level of Processing',
       'gtin_upc', 'brand_owner', 'brand_name', 'branded_food_category',
       'ingredients', 'model_nova'],
      dtype='object')

In [85]:
merged[["Brand Name", "Product Name", "brand_name", "ingredients", "Level of Processing", "model_nova"]]

,Brand Name,Product Name,brand_name,ingredients,Level of Processing,model_nova
0,SILK,"plant milk, soy, flavored",NaN,"SOYMILK (FILTERED WATER, SOYBEANS), CANE SUGAR...",4,4
1,TYSON,"chicken, sausage, patty",Tyson,"Boneless chicken, contains 2% or less of the f...",4,4
2,TYSON,"chicken, drumstick, seasoned",Tyson,"Chicken drumsticks, water, contains 2% or less...",4,4
3,KELLOGG'S,"cracker, graham",NaN,"GRAHAM FLOUR, ENRICHED FLOUR (WHEAT FLOUR, NIA...",4,4
4,KELLOGG'S,"cracker, graham",Kellogg's,"Whole wheat flour, enriched flour (wheat flour...",4,4
5,KELLOGG'S,"cracker, graham",Kellogg's,"Whole wheat flour, enriched flour (wheat flour...",4,4
6,KELLOGG'S,"cracker, graham",Kellogg's,"Whole wheat flour, enriched flour (wheat flour...",4,4
7,PEPPERIDGE FAR,"cracker, whole grain rich",PEPPERIDGE FARM,"MADE WITH SMILES AND WHOLE WHEAT FLOUR, ENRICH...",4,3
8,SWEET BABY RAY,"sauce, barbecue",NaN,"HIGH FRUCTOSE CORN SYRUP, DISTILLED VINEGAR, T...",4,4
9,GENERAL MILLS,"cereal, rice, whole grain rich, ss",NaN,"WHOLE GRAIN RICE, RICE, SUGAR, CANOLA OIL, SAL...",4,4
